In [1]:
%pip install pandas sentence-transformers pinecone-client openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

df = pd.read_csv("Indian_Food_Dataset.csv")
display(df.head())

,TranslatedRecipeName,TranslatedIngredients,TotalTimeInMins,Cuisine,TranslatedInstructions,URL,Cleaned-Ingredients,image-url
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,"salt,amchur (dry mango powder),karela (bitter ...",https://www.archanaskitchen.com/images/archana...
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon m...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,"tomato,salt,chickpea lentils,green chilli,rice...",https://www.archanaskitchen.com/images/archana...
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,"salt,rice vermicelli noodles (thin),asafoetida...",https://www.archanaskitchen.com/images/archana...
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"1/2 teaspoon Turmeric powder (Haldi),1 tablesp...",45,Andhra,To begin making Gongura Chicken Curry Recipe f...,https://www.archanaskitchen.com/gongura-chicke...,"tomato,salt,ginger,sorrel leaves (gongura),fen...",https://www.archanaskitchen.com/images/archana...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"oil - as per use, 1 tablespoon coriander seeds...",30,Andhra,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...,"tomato,salt,ginger,red chillies,curry,asafoeti...",https://www.archanaskitchen.com/images/archana...


In [ ]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "recipe-index"
if index_name not in pc.list_indexes():
    pc.create_index(index_name, dimension=384,spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        ) )
index = pc.Index(index_name)


model = SentenceTransformer('all-MiniLM-L6-v2')
for i, row in df.iterrows():
    combined_text = f"Ingredients: {row['Cleaned-Ingredients']}\nInstructions: {row['TranslatedInstructions']}"
    embedding = model.encode(combined_text)
    metadata = {
        "recipe_name": row['TranslatedRecipeName'],
        "ingredients": row['Cleaned-Ingredients'],
        "measurements": row['TranslatedIngredients'],
        "prep_time": row['TotalTimeInMins'],
        "instructions": row['TranslatedInstructions'],
        "cuisine": row['Cuisine'],
        "url": row['URL'],
        "final_look": row['image-url']
    }
    index.upsert([(str(i), embedding, metadata)])

In [11]:
def query_recipes(user_query, model=SentenceTransformer('all-MiniLM-L6-v2')):
    pc = Pinecone(api_key=PINECONE_API_KEY)
    index = pc.Index("recipe-index")
    
    query_embedding = model.encode(user_query)
    results = index.query(
        vector=query_embedding.tolist(),
        top_k=3,
        include_metadata=True
    )
    metadata_list = [match["metadata"] for match in results.get("matches", [])]
    return metadata_list
    

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
def build_context(metadata_list):
    context = ""
    for metadata in metadata_list:
        context += (
            f"Recipe Name: {metadata.get('recipe_name', 'Unknown')}\n"
            f"Time: {metadata.get('prep_time', 'N/A')}\n"
            f"Ingredients: {metadata.get('ingredients', 'N/A')}\n"
            f"Measurements: {metadata.get('measurements', 'N/A')}\n"
            f"Instructions: {metadata.get('instructions', 'N/A')}\n\n"
        )
    return context

In [6]:
def query_deepseekv3(client, user_input, context):
    # Prepare messages for the conversation
    messages = [
        {"role": "system", "content": "You are a helpful recipe assistant. You are given a list of recipes and a user query. You need to find the most relevant recipe based on the user query."},
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": f"Here are the details I found:\n{context}"}
    ]
    
    # Call the DeepSeekV3 API
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        stream=False  # Set to True for streaming responses
    )
    
    # Extract and return the response content
    return response.choices[0].message.content

In [ ]:
from openai import OpenAI

# Initialize the client
client = OpenAI(
    base_url="https://api.deepseek.com"
)

def recipe_assistant(user_input):
    # Step 1: Retrieve metadata from Pinecone
    results = query_recipes(user_input)  # Implement your Pinecone query logic
    
    if not results:
        return "Sorry, I couldn't find any relevant recipes."

    # Step 2: Build the context string
    context = build_context(results)

    # Step 3: Query DeepSeekV3
    try:
        response = query_deepseekv3(client, user_input, context)
        return response
    except Exception as e:
        return f"Error: {e}"

In [12]:
print(recipe_assistant("What can I cook with chicken and tomato?"))

Based on your query for recipes using chicken and tomato, here are three great options:

1. **Chicken Parmigiana Recipe**  
   - **Time**: 70 minutes  
   - **Key Ingredients**: Chicken breasts, tomato puree, mozzarella cheese, parmesan cheese, garlic, and breadcrumbs.  
   - **Description**: A classic Italian dish where chicken is breaded, fried, and baked with a rich tomato sauce and melted cheese. Perfect served with pasta or a side salad.

2. **Chicken Barbecue Pizza Recipe**  
   - **Time**: 90 minutes  
   - **Key Ingredients**: Chicken breasts, tomato puree, barbecue sauce, mozzarella cheese, and whole wheat flour for the pizza dough.  
   - **Description**: A homemade pizza topped with barbecue chicken, tomato sauce, and melted cheese. Great for a fun dinner or weekend treat.

3. **Tomato Basil Chicken Curry Recipe**  
   - **Time**: 35 minutes  
   - **Key Ingredients**: Chicken breasts, tomato puree, basil leaves, garlic, and carrots.  
   - **Description**: A quick and flavo

In [13]:
print(recipe_assistant("What do I do with leftover rice?"))


If you have leftover rice, here are a few creative and delicious ways to use it:

### 1. **Fried Rice**
   - **Ingredients**: Leftover rice, vegetables (like carrots, peas, bell peppers), soy sauce, eggs, garlic, ginger, and optional protein (chicken, shrimp, or tofu).
   - **Instructions**: Heat oil in a pan, sauté garlic and ginger, add vegetables and protein, scramble eggs, then toss in the rice. Add soy sauce and stir-fry until heated through.

### 2. **Rice Pudding**
   - **Ingredients**: Leftover rice, milk, sugar, cinnamon, vanilla extract, and raisins (optional).
   - **Instructions**: Combine rice, milk, and sugar in a saucepan. Simmer until thickened, stirring occasionally. Add vanilla and cinnamon. Serve warm or chilled.

### 3. **Rice Pancakes**
   - **Ingredients**: Leftover rice, eggs, flour, milk, and a pinch of salt.
   - **Instructions**: Mix rice with eggs, flour, and milk to form a batter. Cook spoonfuls in a greased pan until golden brown on both sides. Serve with s

In [14]:
print(recipe_assistant("Quick Indian food recipes?"))

Here’s a quick and easy Indian recipe based on your query:

### **Vegetable Bajji (Pakora/Fritters)**  
**Time:** 40 minutes  
**Ingredients:**  
- 1 cup Gram flour (besan)  
- 1 Potato (Aloo) - sliced into 1/4 inch pieces  
- 1 Onion - sliced into 1/4 inch pieces  
- 1 Raw Banana - sliced into 1/4 inch pieces  
- 1 teaspoon Red Chilli powder  
- 1 teaspoon Turmeric powder (Haldi)  
- 1/2 teaspoon Cumin powder (Jeera)  
- 1/2 teaspoon Fennel seeds (Saunf) - crushed  
- Salt - to taste  
- Sunflower Oil - for deep frying  

**Instructions:**  
1. Combine all the ingredients for the batter with just enough water to form a thick consistency that can coat the back of a spoon.  
2. Heat oil in a wok or kadai for deep frying.  
3. Coat each of the sliced vegetables in the gram flour batter.  
4. Gradually drop them into the hot oil and deep fry until golden brown.  
5. Drain the Vegetable Bajji on an absorbent paper and serve hot.  

**Serving Suggestion:**  
Serve with Raw Mango & Coconut C

In [15]:

print(recipe_assistant("what can I do with green onions, bell peppers, sweet onions and rice?"))

Based on your ingredients—green onions, bell peppers, sweet onions, and rice—the most relevant recipe is the **Asian Style Mixed Fried Rice**. Here's a simplified version tailored to your ingredients:

### **Asian Style Mixed Fried Rice**
**Ingredients:**
- 2.5 cups cooked rice  
- 1/4 cup sweet onions (or spring onion bulbs), finely chopped  
- 1/4 cup green bell peppers, diced  
- 1/4 cup green onions (spring onion greens), chopped  
- 1 tablespoon sesame oil (or any cooking oil)  
- 1 tablespoon soy sauce  
- 1 teaspoon rice vinegar (optional)  
- Salt and black pepper to taste  

**Instructions:**
1. Heat sesame oil in a wok or large skillet over medium heat.  
2. Add the sweet onions and sauté for 1-2 minutes until softened.  
3. Add the green bell peppers and cook for another 1-2 minutes.  
4. Stir in the cooked rice, soy sauce, and rice vinegar. Mix well to combine.  
5. Add the green onions and season with salt and black pepper. Toss everything together.  
6. Cook for another 2